In [5]:
import os
import urllib3
import pandas as pd
import geopandas as gpd

In [21]:
def download_file(url, path):
    chunk_size = 255

    http = urllib3.PoolManager()
    r = http.request('GET', url, preload_content=False)

    with open(path, 'wb') as out:
        while True:
            data = r.read(chunk_size)
            if not data:
                break
            out.write(data)

    r.release_conn()

In [25]:
DATA_PATH = 'temp_data'

DATASETS = {
  "all_parcel_raw.csv": "https://raw.githubusercontent.com/madison-housing-cs638/data/master/all_parcel_raw.csv",
  "all_parcel_assessments.csv": "https://raw.githubusercontent.com/madison-housing-cs638/data/master/all_parcel_assessments.csv",
  "all_parcel_names.csv": "https://raw.githubusercontent.com/madison-housing-cs638/data/master/all_parcel_names.csv",
  "all_parcel_sales.csv": "https://raw.githubusercontent.com/madison-housing-cs638/data/master/all_parcel_sales.csv",
}


def download_datasets():
  for f in DATASETS:
      filePath = os.path.join(os.pardir, DATA_PATH, f)
      fileUrl = DATASETS[f]
      download_file(fileUrl, filePath)
download_datasets()

In [27]:
def load_csv_datasets(*datasets): #Returns a tupple of pandas dataframes.
    result = []
    for datasetName in datasets:
        fullPath = os.path.join(os.pardir, DATA_PATH, datasetName + '.csv')
        df = pd.read_csv(fullPath)
        result.append(df)
    return tuple(result)


(            Parcel  Year  Land_Assessment  Improvement_Assessment  \
 0                0  2020                0                       0   
 1      60802307997  2019                0                    5000   
 2      70835124359  2019              500                    1500   
 3      60802112023  2019            86200                  201200   
 4      70813307183  2019           114200                  267200   
 ...            ...   ...              ...                     ...   
 80062  81028119320  2019             8700                   67000   
 80063  70816324019  2019            94000                  348800   
 80064  80924427043  2019             8600                  166800   
 80065  70833119154  2019                0                       0   
 80066  70923413987  2019                0                    5000   
 
        Total_Assessment  
 0                     0  
 1                  5000  
 2                  2000  
 3                287400  
 4                38140

In [4]:
city = gpd.read_file("zip://assessment_area.zip")
city.plot()




ImportError: The descartes package is required for plotting polygons in geopandas.